In [1]:
import pandas as pd
import numpy as np
import re
from gensim import corpora, models, similarities

In [2]:
f = '../../data/sherlock_current_wtext.csv'

In [3]:
data = pd.read_csv(f)

In [4]:
df = data

In [5]:
# df = df.head(100)

In [6]:
# df = df.fillna(0)

In [7]:
# df = df.astype(str)

In [8]:
df.columns.values

array(['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks',
       'Category', 'ChapterIndex', 'Chapters', 'Characters', 'Comments',
       'CompleteDate', 'Fandoms', 'Hits', 'Kudos', 'Language', 'Notes',
       'PublishDate', 'Rating', 'Relationship', 'Summary', 'Text', 'Title',
       'UpdateDate', 'Words'], dtype=object)

In [9]:
def bookmarks_values(bms, pubdate):
    if bms != 0:
        pubdate = pubdate[:7]
        v = re.findall(': ([0-9]+)', cell)
        return sum([int(i) for i in v])

In [10]:
df.AdditionalTags = df.AdditionalTags.str.strip()

In [11]:
df.ArchiveWarnings = df.ArchiveWarnings.str.strip()

In [12]:
# df.Bookmarks = df.apply(lambda row:bookmarks_values(row['Bookmarks']),axis = 1)

In [13]:
def ch_values(cell):
    return cell.split('/')[0]

In [14]:
df.Chapters = df.apply(lambda row:ch_values(row['Chapters']),axis = 1)

In [15]:
df.Characters = df.Characters.str.strip()

In [16]:
def comments_values(cell):
    if cell != 0:
        v = re.findall(': ([0-9]+)', cell)
        return sum([int(i) for i in v])

In [17]:
# df.Comments = df.apply(lambda row:comments_values(row['Comments']),axis = 1)

In [18]:
df.Rating = df.Rating.str.strip()

In [19]:
df.Relationship = df.Relationship.str.strip()

In [20]:
df['FullLength'] = df['Words']

In [21]:
df['ChapterLength'] = len(df.Text.str.split()[0])

In [22]:
lda = models.LdaModel.load('../20160225_lda/sherlock_40topics')

In [23]:
def get_gamma(text, id2word):
    #pre-processing
    #input: a slice of the original df
    #output: a list of lists each containing cleaned words from a work
    text = text.lower().split()
    text = [re.sub(r'[^A-Za-z0-9]+', '', str(word)) for word in text]
    text = [word for word in text if len(word) > 1 and not word.isdigit()]
    gamma = lda.get_document_topics(id2word.doc2bow(text), minimum_probability = 0)
    return gamma

In [24]:
def preprocess(df):
    #pre-processing
    #input: a slice of the original df
    #output: a list of lists each containing cleaned words from a work
    df = df.astype(str)
    text = df[['Text']]
    text = text.drop_duplicates().Text.str.lower().str.split()
    text_cleaned = []
    for line in text:
        line = [re.sub(r'[^A-Za-z0-9]+', '', str(word)) for word in line]
        line = [word for word in line if len(word) > 1 and not word.isdigit()]
        text_cleaned.append(line)
    return text_cleaned

In [25]:
text_all = preprocess(df)
print len(text_all)
id2word = corpora.dictionary.Dictionary(text_all)

In [27]:
import cPickle as pickle
id2word = pickle.load(open('../../id2word.p'))

In [28]:
id2word[10]

u'wellxc2xa0just'

In [29]:
df['TopicDistribution'] = df.apply(lambda row:get_gamma(row['Text'], id2word), axis=1)

In [30]:
df.head()

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,ChapterIndex,Chapters,Characters,Comments,CompleteDate,...,Rating,Relationship,Summary,Text,Title,UpdateDate,Words,FullLength,ChapterLength,TopicDistribution
0,"Implied/Referenced Rape/Non-con, Past Rape/Non...",Creator Chose Not To Use Archive Warnings,HunterusHeroicus,{'2016-03': 1},Gen,NaN,1,"John Watson, Sherlock Holmes, Mycroft Holmes, ...",{'2016-01': 2},2015-01-11,...,Mature,NaN,"Mycroft pays a visit, and John gets some answers.","\nWhen John got back to the flat, his head was...",In Memoriam,NaN,1016,1016,974,"[(0, 2.66524520256e-05), (1, 0.0268015104776),..."
1,Implied/Referenced Rape/Non-con,Creator Chose Not To Use Archive Warnings,HunterusHeroicus,NaN,Gen,NaN,1,"John Watson, Greg Lestrade",NaN,2015-01-11,...,Mature,NaN,A conversation with Lestrade.,\nJohn knocked on the door before entering. Le...,Puzzle Pieces,NaN,464,464,974,"[(0, 5.80046403712e-05), (1, 0.0650570155247),..."
2,Implied/Referenced Rape/Non-con,Creator Chose Not To Use Archive Warnings,HunterusHeroicus,NaN,Gen,NaN,1,"Molly Hooper, John Watson, Sherlock Holmes",NaN,2015-01-11,...,Mature,NaN,John had almost gotten Sherlock out the door w...,\nJohn had almost gotten Sherlock out the door...,"Molly, Interrupted",NaN,303,303,974,"[(0, 8.77192982456e-05), (1, 8.77192982456e-05..."
3,"Crack, Seriously crack, Cannibalism, but not g...",No Archive Warnings Apply,i_ship_an_armada,"{'2015-02': 1, '2015-12': 1, '2016-03': 1, '20...",M/M,NaN,1,"Sherlock Holmes, John Watson","{'2015-09': 2, '2015-05': 2, '2015-01': 35}",2015-01-11,...,Teen And Up Audiences,Sherlock Holmes/John Watson,John is suddenly aware of Sherlock\xe2\x80\x99...,\nA whip cracks. \xc2\xa0John looks up at the ...,Put That on Your Blog,NaN,1028,1028,974,"[(0, 0.420445258288), (1, 2.65111346766e-05), ..."
4,"Rape/Non-con Elements, Past Rape/Non-con",Creator Chose Not To Use Archive Warnings,HunterusHeroicus,NaN,Gen,NaN,1,"John Watson, Sherlock Holmes, Jim Moriarty",NaN,2015-01-11,...,Mature,NaN,John knew it was only a matter of time before ...,"\n""Look, all I\'m saying is that\'s not kind t...",Encounter,NaN,958,958,974,"[(0, 2.66524520256e-05), (1, 0.0601878095899),..."


In [31]:
df2 = df[['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks', 'Category', 'Chapters', 'Characters', 'Comments',\
       'CompleteDate', 'Fandoms', 'Hits', 'Kudos', 'PublishDate', 'Rating', 'Relationship', 'TopicDistribution', 'FullLength', 'ChapterLength']]

In [32]:
df2.head(1)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Comments,CompleteDate,Fandoms,Hits,Kudos,PublishDate,Rating,Relationship,TopicDistribution,FullLength,ChapterLength
0,"Implied/Referenced Rape/Non-con, Past Rape/Non...",Creator Chose Not To Use Archive Warnings,HunterusHeroicus,{'2016-03': 1},Gen,1,"John Watson, Sherlock Holmes, Mycroft Holmes, ...",{'2016-01': 2},2015-01-11,Sherlock (TV),254,5,2015-01-11,Mature,NaN,"[(0, 2.66524520256e-05), (1, 0.0268015104776),...",1016,974


In [34]:
df2.to_csv('../../data/sherlock_processed.tsv', index = False, sep = '\t')